### Libraries

In [46]:
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
import datetime

'machine learning library imports'
from sklearn import tree
from sklearn import metrics
from sklearn.utils import shuffle

### Load data

In [47]:
AUGMENTATION = 'SMOTE'

data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mqtt.topic_0.0.1', 
                                                       'Unnamed: 0', 
                                                       'mqtt.topic_Temperature_and_Humidity', 
                                                       'mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])                                    

Loading complete.
Training data: 1500000 rows, 98 columns. 
Test data: 381934 rows, 98 columns.


### Data preparation

In [48]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [49]:
# One-hot encode the training and test labels if needed
X_train_enc, X_test_enc = utils.one_hot_encode_categorical(X_train, X_test, random_state=42)

No categorical features found. Returning original datasets.


#### Label encoding

In [58]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model Training

In [60]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Instantiate the decision tree classifier
dt = tree.DecisionTreeClassifier(random_state=42)

# Train the model
dt_clf = dt.fit(X_train, y_train)

### Model Evaluation

In [72]:
predictions = dt_clf.predict(X_test)
predictions[:3]

array(['DDoS_HTTP', 'DDoS_HTTP', 'DDoS_UDP'], dtype=object)

In [73]:
# encode the predictions
predictions_enc = le.transform(predictions)
predictions_enc[:3]

array([1, 1, 4])

In [75]:
# Calculate metrics
accuracy = metrics.accuracy_score(y_test, predictions)
precision_w = metrics.precision_score(y_test, predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(y_test, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test, predictions, average='weighted')
precision_m = metrics.precision_score(y_test, predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(y_test, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test, predictions, average='macro')

print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision (macro): {:.2f}".format(precision_m))
print("Recall (macro): {:.2f}".format(recall_m))
print("F1 (macro): {:.2f}".format(f1_score_m))
print("Precision (weighted): {:.2f}".format(precision_w))
print("Recall (weighted): {:.2f}".format(recall_w))
print("F1 (weighted): {:.2f}".format(f1_score_w))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")

Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.10
Precision (macro): 0.74
Recall (macro): 0.20
F1 (macro): 0.08
Precision (weighted): 0.92
Recall (weighted): 0.10
F1 (weighted): 0.07
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [76]:
# create dictionary for results
results = {
    "model": "Decision Tree",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/dt.csv')

#### Confusion Matrix

In [78]:
conf_mat = metrics.confusion_matrix(y_test, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,0,2568,0,0,0,0,0,0,0,0,0,0,0,2145,69
DDoS_HTTP,0,2675,0,0,0,0,0,0,0,0,0,0,0,6290,663
DDoS_ICMP,0,0,0,0,0,0,0,0,0,0,0,0,0,13501,0
DDoS_TCP,0,9385,0,0,0,0,0,0,0,0,0,0,0,624,0
DDoS_UDP,0,0,0,0,24592,0,0,0,0,0,0,0,0,9,0
Fingerprinting,0,47,0,0,0,0,0,0,0,0,0,0,0,98,1
MITM,0,71,0,0,0,0,0,0,0,0,0,0,0,5,0
Normal,0,111746,0,0,185,0,0,248,0,0,0,0,0,107834,52763
Password,0,3762,0,0,0,0,0,0,0,0,0,0,0,3415,2931
